In [ ]:
from langchain_community.llms import Ollama
import numpy as np
model="phi4:latest" 
# Initialize the Ollama LLM
llm = Ollama(model = model , temperature= 0.7)
# llm = Ollama(model = model , temperature= 0 , base_url="http://192.168.0.10:11434")



In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser




# JSON parser
parser = JsonOutputParser()

# Persian language enforced prompt
prompt = PromptTemplate.from_template("""
You are a helpful assistant.

There are two contexts provided below.

Context A:
{context_a}

Context B:
{context_b}

Your task:
1. Generate one question that is directly relevant to Context A and provide an accurate response based on Context A.
2. Generate another question that is not related to Context A but is relevant to Context B.

Important: All output must be in Persian (Farsi) language.

Return the result strictly in the following JSON format (but translated to Persian):

{{
  "relevant_question_to_context_a": "...",
  "answer_based_on_context_a": "...",
  "irrelevant_question_to_context_a_but_relevant_to_context_b": "..."
}}
""")


# Define chain
chain = (
    RunnableMap({
        "context_a": RunnablePassthrough(),
        "context_b": RunnablePassthrough()
    })
    | prompt
    | llm
    | parser
)




In [ ]:
import pandas as pd
data = pd.read_excel("question.xlsx").to_numpy()
dataset = []
for i in data:
    dataset.append(i[0] + " " + i[1])

In [ ]:
final_dataset = []
for _ in range(30):
    for i in range(len(dataset)):
        rand = np.random.randint(0 , len(dataset))
        result = chain.invoke({
            "context_a": dataset[i],
            "context_b": dataset[rand]})
        
        final_dataset.append({"anchor":result.get("answer_based_on_context_a"),
                             "positive":result.get("relevant_question_to_context_a"),
                             "negetive":result.get("irrelevant_question_to_context_a_but_relevant_to_context_b")})


In [ ]:
from datasets import Dataset


dataset = Dataset.from_list(final_dataset)


print(dataset)
print(dataset[0])


Dataset({
    features: ['name', 'age', 'city'],
    num_rows: 3
})
{'name': 'Ali', 'age': 28, 'city': 'Tehran'}
